In [ ]:
import polars as pl
import validate.lesion as vl
import validate.general as vg

from constants import list_id_cols

import utils
from pathlib import Path
import os

PATH_INPUT = os.getenv("PATH_INPUT")

lf = utils.get_df(Path(PATH_INPUT).joinpath("test data.accdb")).lazy()

lf2 = utils.get_df(Path(PATH_INPUT).joinpath("ZON CHERAS OC DATABASE JULY-OCT 2023 edit MAIwani.accdb")).lazy()

obj = vg.ValidationGeneral(lf, "test data")
lfall = pl.concat([lf, lf2])

In [ ]:
lfall.columns

In [ ]:
famihistcancer_whitelist = ["false|false|false", "true|true|true"]
additionaldetails_blacklist = ["true|false|true", "true|true|false"]

lfall.select("LESION", "OCCUPATION", "EDUCATION LEVEL CODE").with_columns(
  pl.when(pl.col("OCCUPATION").is_not_null())
  .then(True)
  .otherwise(False)
  .alias("occupation_filled"),
  pl.when(pl.col("EDUCATION LEVEL CODE").is_not_null())
  .then(True)
  .otherwise(False)
  .alias("education_filled"),
).with_columns(
  pl.concat_str(
    ["LESION", "occupation_filled", "education_filled"], separator="|"
  ).alias("invalid_combination")
).filter(pl.col("invalid_combination").is_in(additionaldetails_blacklist)).collect()
